In [1]:
%load_ext tensorboard

In [2]:
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import datetime, os, shutil


In [3]:
num_inputs=8
num_classes=5

model_names=["rosenblatt", "hebb", "samuel"]

In [4]:
data=pd.read_csv(f"./dataset/stress-level.csv")
data.head()

,sr,rr,t,lm,bo,rem,sr.1,hr,sl
0,93.80,25.680,91.840,16.600,89.840,99.60,1.840,74.20,3
1,91.64,25.104,91.552,15.880,89.552,98.88,1.552,72.76,3
2,60.00,20.000,96.000,10.000,95.000,85.00,7.000,60.00,1
3,85.76,23.536,90.768,13.920,88.768,96.92,0.768,68.84,3
4,48.12,17.248,97.872,6.496,96.248,72.48,8.248,53.12,0


In [5]:
data.shape

(630, 9)

In [6]:
data.columns=['snoring_rate', 
              'respiration_rate', 
              'body_temperature', 
              'limb_movement', 
              'blood_oxygen', 
              'eye_movement', 
              'sleeping_hours', 
              'heart_rate', 
              'stress_level'
             ]

In [7]:
data.isnull().any()

snoring_rate        False
respiration_rate    False
body_temperature    False
limb_movement       False
blood_oxygen        False
eye_movement        False
sleeping_hours      False
heart_rate          False
stress_level        False
dtype: bool

In [8]:
data.describe().T

,count,mean,std,min,25%,50%,75%,max
snoring_rate,630.0,71.6,19.372833,45.0,52.50,70.0,91.25,100.0
respiration_rate,630.0,21.8,3.966111,16.0,18.50,21.0,25.00,30.0
body_temperature,630.0,92.8,3.529690,85.0,90.50,93.0,95.50,99.0
limb_movement,630.0,11.7,4.299629,4.0,8.50,11.0,15.75,19.0
blood_oxygen,630.0,90.9,3.902483,82.0,88.50,91.0,94.25,97.0
eye_movement,630.0,88.5,11.893747,60.0,81.25,90.0,98.75,105.0
sleeping_hours,630.0,3.7,3.054572,0.0,0.50,3.5,6.50,9.0
heart_rate,630.0,64.5,9.915277,50.0,56.25,62.5,72.50,85.0
stress_level,630.0,2.0,1.415337,0.0,1.00,2.0,3.00,4.0


In [9]:
x_train, x_test, y_train, y_test = train_test_split(
    data.iloc[:, :num_inputs], 
    data['stress_level'], 
    test_size=0.2, 
    random_state=num_inputs*num_inputs
)

In [10]:
y_train=to_categorical(y_train, num_classes)
y_test=to_categorical(y_test, num_classes)
y_train[:num_classes]

array([[0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1.],
       [1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0.]], dtype=float32)

In [11]:
def configure_model(config_id):
    
    if config_id == 1 :
        return tf.keras.models.Sequential([
            tf.keras.Input(shape=(8,)),
            tf.keras.layers.Dense(16, activation='sigmoid'),
            tf.keras.layers.Dense(5, activation='softmax')
          ])
    elif config_id == 2 :
        return tf.keras.models.Sequential([
            tf.keras.Input(shape=(8,)),
            tf.keras.layers.Dense(64, activation='relu'),
            tf.keras.layers.Dense(32, activation='relu'),
            tf.keras.layers.Dense(5, activation='softmax')
          ])
    elif config_id == 3 :
        return tf.keras.models.Sequential([
            tf.keras.Input(shape=(8,)),
            tf.keras.layers.Dense(16, activation='tanh'),
            tf.keras.layers.Dense(10, activation='tanh'),
            tf.keras.layers.Dense(5, activation='softmax')
          ])
    else :
        raise ValueError("Provided config_id does not match any defined configuration")
  

In [12]:
def create_model(config_id):
    model = configure_model(config_id)

    model.compile(optimizer='adam',
      loss='mean_squared_error',
      metrics=['accuracy'])

    return model

  

In [13]:
def train_model(config_id):
    model = create_model(config_id)

    model.summary()

    logdir = os.path.join("logs", model_names[config_id - 1])

    shutil.rmtree(logdir)

    tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

    model.fit(
      x=x_train,
      y=y_train,
      epochs=75,
      validation_data=(x_test, y_test),
      callbacks=[tensorboard_callback]
    )

    model.save('models/' + model_names[config_id - 1] + '_model.h5')

In [14]:
%tensorboard --logdir logs --bind_all

In [32]:
train_model(1) #Rosenblatt

Model: "sequential_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_43 (Dense)            (None, 16)                144       
                                                                 
 dense_44 (Dense)            (None, 5)                 85        
                                                                 
Total params: 229
Trainable params: 229
Non-trainable params: 0
_________________________________________________________________
Epoch 1/75
16/16 [==============================] - 0s 9ms/step - loss: 0.1631 - accuracy: 0.2163 - val_loss: 0.1644 - val_accuracy: 0.1984
Epoch 2/75
16/16 [==============================] - 0s 4ms/step - loss: 0.1564 - accuracy: 0.3234 - val_loss: 0.1588 - val_accuracy: 0.3651
Epoch 3/75
16/16 [==============================] - 0s 4ms/step - loss: 0.1521 - accuracy: 0.4087 - val_loss: 0.1503 - val_accuracy: 0.3651
Epoch 4/75
16/16 [=============

16/16 [==============================] - 0s 4ms/step - loss: 0.0883 - accuracy: 0.8016 - val_loss: 0.0875 - val_accuracy: 0.7937
Epoch 55/75
16/16 [==============================] - 0s 4ms/step - loss: 0.0875 - accuracy: 0.8016 - val_loss: 0.0868 - val_accuracy: 0.7937
Epoch 56/75
16/16 [==============================] - 0s 4ms/step - loss: 0.0868 - accuracy: 0.8016 - val_loss: 0.0858 - val_accuracy: 0.7937
Epoch 57/75
16/16 [==============================] - 0s 4ms/step - loss: 0.0860 - accuracy: 0.8016 - val_loss: 0.0850 - val_accuracy: 0.7937
Epoch 58/75
16/16 [==============================] - 0s 4ms/step - loss: 0.0854 - accuracy: 0.8016 - val_loss: 0.0843 - val_accuracy: 0.7937
Epoch 59/75
16/16 [==============================] - 0s 4ms/step - loss: 0.0846 - accuracy: 0.8016 - val_loss: 0.0836 - val_accuracy: 0.7937
Epoch 60/75
16/16 [==============================] - 0s 4ms/step - loss: 0.0840 - accuracy: 0.8016 - val_loss: 0.0833 - val_accuracy: 0.7937
Epoch 61/75
16/16 [======

In [24]:
train_model(2) # Hebb

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_26 (Dense)            (None, 64)                576       
                                                                 
 dense_27 (Dense)            (None, 32)                2080      
                                                                 
 dense_28 (Dense)            (None, 5)                 165       
                                                                 
Total params: 2,821
Trainable params: 2,821
Non-trainable params: 0
_________________________________________________________________
Epoch 1/75
16/16 [==============================] - 0s 10ms/step - loss: 0.2559 - accuracy: 0.2401 - val_loss: 0.2234 - val_accuracy: 0.1746
Epoch 2/75
16/16 [==============================] - 0s 6ms/step - loss: 0.1723 - accuracy: 0.4266 - val_loss: 0.1483 - val_accuracy: 0.5873
Epoch 3/75
16/16 [=================

16/16 [==============================] - 0s 5ms/step - loss: 0.1209 - accuracy: 0.6032 - val_loss: 0.1252 - val_accuracy: 0.5873
Epoch 54/75
16/16 [==============================] - 0s 5ms/step - loss: 0.1207 - accuracy: 0.6032 - val_loss: 0.1251 - val_accuracy: 0.5873
Epoch 55/75
16/16 [==============================] - 0s 5ms/step - loss: 0.1210 - accuracy: 0.6032 - val_loss: 0.1244 - val_accuracy: 0.5873
Epoch 56/75
16/16 [==============================] - 0s 5ms/step - loss: 0.1199 - accuracy: 0.6032 - val_loss: 0.1247 - val_accuracy: 0.5873
Epoch 57/75
16/16 [==============================] - 0s 5ms/step - loss: 0.1202 - accuracy: 0.6032 - val_loss: 0.1267 - val_accuracy: 0.5873
Epoch 58/75
16/16 [==============================] - 0s 5ms/step - loss: 0.1213 - accuracy: 0.6032 - val_loss: 0.1300 - val_accuracy: 0.5873
Epoch 59/75
16/16 [==============================] - 0s 5ms/step - loss: 0.1214 - accuracy: 0.6032 - val_loss: 0.1261 - val_accuracy: 0.5873
Epoch 60/75
16/16 [======

In [23]:
train_model(3) # Samuel

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_23 (Dense)            (None, 16)                144       
                                                                 
 dense_24 (Dense)            (None, 10)                170       
                                                                 
 dense_25 (Dense)            (None, 5)                 55        
                                                                 
Total params: 369
Trainable params: 369
Non-trainable params: 0
_________________________________________________________________
Epoch 1/75
16/16 [==============================] - 0s 9ms/step - loss: 0.1773 - accuracy: 0.1647 - val_loss: 0.1459 - val_accuracy: 0.3413
Epoch 2/75
16/16 [==============================] - 0s 5ms/step - loss: 0.1385 - accuracy: 0.4048 - val_loss: 0.1344 - val_accuracy: 0.3810
Epoch 3/75
16/16 [======================

16/16 [==============================] - 0s 5ms/step - loss: 0.0035 - accuracy: 1.0000 - val_loss: 0.0036 - val_accuracy: 1.0000
Epoch 54/75
16/16 [==============================] - 0s 4ms/step - loss: 0.0035 - accuracy: 1.0000 - val_loss: 0.0038 - val_accuracy: 1.0000
Epoch 55/75
16/16 [==============================] - 0s 4ms/step - loss: 0.0033 - accuracy: 1.0000 - val_loss: 0.0033 - val_accuracy: 1.0000
Epoch 56/75
16/16 [==============================] - 0s 5ms/step - loss: 0.0030 - accuracy: 1.0000 - val_loss: 0.0037 - val_accuracy: 1.0000
Epoch 57/75
16/16 [==============================] - 0s 5ms/step - loss: 0.0030 - accuracy: 1.0000 - val_loss: 0.0042 - val_accuracy: 1.0000
Epoch 58/75
16/16 [==============================] - 0s 4ms/step - loss: 0.0028 - accuracy: 1.0000 - val_loss: 0.0028 - val_accuracy: 1.0000
Epoch 59/75
16/16 [==============================] - 0s 5ms/step - loss: 0.0025 - accuracy: 1.0000 - val_loss: 0.0027 - val_accuracy: 1.0000
Epoch 60/75
16/16 [======